In [1]:
%run input/Format.ipynb
!rm output/dynamic_delta_fits/?/*
import ROOT as root
from array import array
from math import sin
from ROOT import gRandom
from ROOT import TFile
from ROOT import TPad, TLegend
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Welcome to JupyROOT 6.30/06


In [2]:
iOption0 = 0
is_sigmalised = 0
sim = 0
new = 1

Min_Entries = 100

In [3]:
isigma = 2
N_files = 1
N_layers = 8
N_hists = 4
N_phi = 6
N_ilyaers = 1
is_e = 1
layers = ["1-2 layer pos","1-2 layer neg","2-3 layer pos","2-3 layer neg","2-4 layer pos","2-4 layer neg","3-4 layer pos","3-4 layer neg","20-40%","40-60%","60-80%","80-93%"]
phi_bins = [35,36,40,41,47,48,52,53,59,60,64,65]
phi_bins = [91,91,96,96,100,100,101,101,105,105,110,110]
if is_sigmalised: phi_bins = [45,45,46,46,48,50,51,53,55,55,56,56]
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
file_path="input/rg/"
file_names="my-27th_QA_19200_350runs" #my-25th_QA_19197_1000runs
if new:
    file_path="input/NewHadron/"
    file_names="missalign_hadron_new6_rg0-8"
hist_select_3D_names = ["dphi_hist","dthe_hist","sdphi_hist","sdthe_hist"]
if is_e == 1: hist_select_3D_names = ["dphi_hist_el_dynamic","dthe_hist_el_dynamic","sdphi_hist_el_dynamic","sdthe_hist_el_dynamic"]
iOptames=["phi","the"]

In [4]:
if sim :
    file_path="../sim/"
    file_names="my-kek"

    #phi_bins = [96,97,98,99,100,100,101,101,102,103,104,105]

In [5]:
hists_emc_read = []

for iFile in range(N_files):
    print(file_path+file_names+f"{iFile}.root")
    if not new: infile = root.TFile.Open(file_path+file_names+f"{iFile}.root", "read")
    else: infile = root.TFile.Open(file_path+file_names+f".root", "read")
    hists_emc_read0 = []
    for ilayer in range(N_layers):
        hist_select_3D0 = []
        for ihist in range(len(hist_select_3D_names)):
            hist_select_3D0.append(infile.Get(hist_select_3D_names[ihist]+f"_{ilayer}"))
            hist_select_3D0[-1].SetDirectory(root.nullptr)
        hists_emc_read0.append(hist_select_3D0)
    hists_emc_read.append(hists_emc_read0)

infile.Close()

input/NewHadron/missalign_hadron_new6_rg0-80.root


In [6]:
pt = array( 'd' )
pt_boarders = array( 'd', [0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.4,2.0,2.8,4.9])
if is_sigmalised: pt_boarders = array( 'd', [0.2,0.5,0.8,1.2,1.6,2.0,2.4,3.0,3.8,4.9])
pt_err = array( 'd' )
N_pt = len(pt_boarders) - 1
for i in range(N_pt):
    pt.append( (pt_boarders[i]+pt_boarders[i+1])/2)
    pt_err.append((pt_boarders[i+1]-pt_boarders[i])/2)
print("pt = ", pt,"\npt_boarders = ",pt_boarders)
if not is_sigmalised:  pt[0]=0.225

pt =  array('d', [0.25, 0.35, 0.45, 0.55, 0.6499999999999999, 0.75, 0.8500000000000001, 0.95, 1.2, 1.7, 2.4, 3.85]) 
pt_boarders =  array('d', [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.4, 2.0, 2.8, 4.9])


In [7]:
boards_tot = [[-0.099,0.099],[-0.099,0.099]]
boards_fit = [[-0.011,0.012],[-0.011,0.012]]
boards_fit2 = [[-0.03,-0.01],[0.015,0.035],[-0.09,-0.03],[0.035,0.09],[-0.09,-0.03],[0.035,0.09],[-0.06,-0.025],[0.03,0.065]]
boards_fit3 = [[-0.02,0.02],[-0.02,0.02]]
if is_sigmalised==1:
    boards_tot = [[-5,5],[-5,5]]
    boards_fit = [[-1,1],[-1,1]]
if is_sigmalised == 1:
    boards_tot = [[-10,10],[-10,10]]
    boards_fit = [[-1,1],[-1,1]]
    boards_fit2 = [[-6,-3],[3,6],[-6,-2],[2,6],[-6,-2],[2,6],[-6,-2],[2,6]]
    boards_fit3 = [[3,7],[-7,-3]]

In [8]:
def emc_fitter(InHist, iphi, seed=1, ilayer=0, sigma=2, fits_mean=0, fits_sigma=0):
    N_gaus=2
    seed0 = seed
    seed=seed*N_pt*ilayer*100

    hist_pt_bin = InHist.ProjectionZ("kek")

    pt_local =  array( 'd' )
    pt_local_err =  array( 'd' )

    central_bean_prev = (0.001*phi_bins[2*iphi]-0.0005-0.1)
    if is_sigmalised : central_bean_prev = 0
    Mean_local =  array( 'd' )
    Sigma_local =  array( 'd' )
    Mean_local_err =  array( 'd' )
    Sigma_local_err =  array( 'd' )

    int_fg_local =  array( 'd' )
    int_bg_local =  array( 'd' )
 
    for p in range(N_pt):
        c1 = root.TCanvas(f"c1{seed+p}",f"c1{seed+p}",720,360)

        m2_name=str(ilayer)+"/"+str(iphi)+"dynamic_"+str(round(pt[p],1))

        emc_var_porj = InHist.ProjectionX(f"emc{seed+p}_{seed+p}_{ilayer}_{iphi}",phi_bins[2*iphi],phi_bins[2*iphi],hist_pt_bin.FindBin(pt_boarders[p]),hist_pt_bin.FindBin(pt_boarders[p+1])-1)
        emc_var_porj.SetLineColor(1)

        if emc_var_porj.GetEntries() < Min_Entries:
            continue

        pt_loc = 0.5* (pt_boarders[p]+pt_boarders[p+1])
        pt_loc_err = (pt_boarders[p+1]-pt_boarders[p])/4.

        pt_local.append(pt_loc)
        pt_local_err.append(pt_loc_err)

        par_tot = array( 'd', (3*N_gaus+3)*[2.] )
        g_gaus= root.TF1( f'g_gaus_{seed+p}_{ilayer}_{iphi}', 'gaus',  boards_fit[int(ilayer%2)][0]+central_bean_prev, boards_fit[int(ilayer%2)][1]+central_bean_prev)
        g_gaus2= root.TF1( f'g_gaus_2_{seed+p}_{ilayer}_{iphi}', 'gaus',  boards_fit2[ilayer][0]+central_bean_prev, boards_fit2[ilayer][1]+central_bean_prev)#[0]/[2]/sqrt(2*3.14159)*exp(-0.5*pow((x-[1])/[2],2))
        g_pol = root.TF1( f'g_pol{seed+p}_{ilayer}_{iphi}', '[0]+[1]*x*(abs(x)/x-1)+[2]*x*(abs(x)/x+1)',  boards_tot[int(ilayer%2)][0],boards_tot[int(ilayer%2)][1])
        Format_Func(g_pol,3,2)

        total = root.TF1( f'total_pos{seed+p}_{ilayer}_{iphi}', 'gaus(0)+gaus(3)+[6]+[7]*x*(abs(x)/x-1)+[8]*x*(abs(x)/x+1)',boards_tot[int(ilayer%2)][0], boards_tot[int(ilayer%2)][1])
        Format_Func(total,2,6)

        g_gaus.SetParameter(1,0.)
        g_gaus.SetParameter(2,boards_fit[1][1])
        g_gaus2.SetParameter(1,0.5*(boards_fit2[ilayer][0]+boards_fit2[ilayer][1])+central_bean_prev)
        g_gaus2.SetParameter(2,0.5*(boards_fit2[ilayer][1]-boards_fit2[ilayer][0]))
        g_gaus2.SetParLimits(0,1,10000000)
        g_gaus2.SetParLimits(1,boards_fit2[ilayer][0]+central_bean_prev, boards_fit2[ilayer][1]+central_bean_prev)

        emc_var_porj.Draw()
        emc_var_porj.Fit( g_gaus, 'QR')

        if iOption0 == 4:
            Mean_local.append(g_gaus.GetParameter(1))
            Sigma_local.append(g_gaus.GetParameter(2))
            Mean_local_err.append(g_gaus.GetParError(1)+0.05*g_gaus.GetParameter(2))
            Sigma_local_err.append(g_gaus.GetParError(2)+0.05*g_gaus.GetParameter(2))
            c1.SaveAs( "output/d_delta_fits/" + m2_name + ".png" )
            continue

        emc_var_porj.Fit( g_gaus2, 'QR+' )
        emc_var_porj.Fit( g_pol, 'QR+' )

        par_gaus = g_gaus.GetParameters()
        par_gaus2 = g_gaus2.GetParameters()
        par_pol = g_pol.GetParameters()

        par_tot[0], par_tot[1], par_tot[2], par_tot[3], par_tot[4], par_tot[5] = par_gaus[0], par_gaus[1], par_gaus[2], par_gaus2[0], par_gaus2[1], par_gaus2[2]
        par_tot[6], par_tot[7], par_tot[8] = par_pol[0], par_pol[1], par_pol[1]

        total.SetParameters( par_tot )
        total.SetParLimits(1,boards_tot[int(ilayer%2)][0]/10+central_bean_prev,boards_tot[int(ilayer%2)][1]/10+central_bean_prev)
        total.SetParLimits(2,boards_fit[int(ilayer%2)][1]/10,2)
        total.SetParLimits(3,par_gaus2[0]*0.49, par_gaus2[0]*2)
        total.SetParLimits(4,par_gaus2[1]-0.01, par_gaus2[1]+0.01)
        total.SetParLimits(5,par_gaus2[2]*0.49, par_gaus2[2]*2)
        #total.SetParLimits(3,1,1000000)
        if iphi>-1:
            total.SetParLimits(1,boards_tot[int(ilayer%2)][0],boards_tot[int(ilayer%2)][1])
        #if iOption0%2==1: total.FixParameter(3,0.)
        
        emc_var_porj.Draw()
        emc_var_porj.Fit( total, 'QR' )

        par_tot = total.GetParameters()

        g_gaus= root.TF1( f'g_gaus_4_{seed+p}_{ilayer}_{iphi}', 'gaus',  par_tot[1]-2*par_tot[2], par_tot[1]+2*par_tot[2])
        g_gaus2= root.TF1( f'g_gaus_5_{seed+p}_{ilayer}_{iphi}', 'gaus',  par_tot[4]-2*par_tot[5], par_tot[4]+2*par_tot[5])

        Format_Func(g_gaus,3,4)
        Format_Func(g_gaus2,3,root.kGreen+2)

        par_gaus[0], par_gaus[1], par_gaus[2], par_gaus2[0], par_gaus2[1], par_gaus2[2] = par_tot[0], par_tot[1], par_tot[2], par_tot[3], par_tot[4], par_tot[5]
        par_pol[0], par_pol[1], par_pol[2] = par_tot[6], par_tot[7], par_tot[8]
        g_gaus2.SetParameters(par_gaus2)
        g_gaus.SetParameters(par_gaus)
        g_pol.SetParameters(par_pol)

        g_gaus.Draw('same')
        g_gaus2.Draw('same')
        g_pol.Draw('same')

        Mean_local.append(total.GetParameter(1))
        Sigma_local.append(total.GetParameter(2))
        if iOption0<4 and is_sigmalised==0:
            Mean_local_err.append(total.GetParError(1)+0.05*total.GetParameter(2))
            Sigma_local_err.append(total.GetParError(2)+0.05*total.GetParameter(2))
        else:
            Mean_local_err.append(total.GetParError(1)+ (0.15-0.005*p)*total.GetParameter(2))
            Sigma_local_err.append(total.GetParError(2)+(0.15-0.005*p)*total.GetParameter(2))


        int_fg_local.append(g_gaus.Integral(total.GetParameter(1)-sigma*total.GetParameter(2),total.GetParameter(1)+sigma*total.GetParameter(2)))
        int_bg_local.append(g_pol.Integral(total.GetParameter(1)-sigma*total.GetParameter(2),total.GetParameter(1)+sigma*total.GetParameter(2)))
            
        #m2_pos.Draw("same") m2_neg.Draw("same")

        c1.SaveAs( "output/dynamic_delta_fits/" + m2_name + ".png" )

    return [Mean_local, Mean_local_err, Sigma_local, Sigma_local_err, int_fg_local, int_bg_local, pt_local, pt_local_err]


In [9]:
def emc_centr_fitter(InHist, layers, seed=1, sigma=2, fits_mean=0, fits_sigma=0):
    out_hists = []
    for ilayer in range(N_layers):
        out_hists1 = []
        for iphi in layers:
            out_hists1.append(emc_fitter(InHist[seed][ilayer][iOption0+is_sigmalised*2], iphi, seed, ilayer, sigma, fits_mean, fits_sigma))
        out_hists.append(out_hists1)
    return out_hists

In [10]:
import multiprocess as mp

Ntr = N_files

pool = mp.Pool(Ntr)
output_array = pool.starmap(emc_centr_fitter, [(hists_emc_read, [0,1,2,3,4,5], i, isigma) for i in range(Ntr)])
pool.close()

In [11]:
Means, Mean_errs, Sigmas, Sigma_errs, Int_FG, Int_BG, pt_arr, pt_arr_err = [], [], [], [], [], [], [], []
for ifile in range(Ntr):
    Means1, Mean_errs1, Sigmas1, Sigma_errs1, Int_FG1, Int_BG1, pt_arr1, pt_arr_err1 = [], [], [], [], [], [], [], []
    for ilayer in range(N_layers):
        Means0, Mean_errs0, Sigmas0, Sigma_errs0, Int_FG0, Int_BG0, pt_arr0, pt_arr_err0 = [], [], [], [], [], [], [], []
        for iphi in range(N_phi):
            Means0.append(output_array[ifile][ilayer][iphi][0])
            Mean_errs0.append(output_array[ifile][ilayer][iphi][1])
            Sigmas0.append(output_array[ifile][ilayer][iphi][2])
            Sigma_errs0.append(output_array[ifile][ilayer][iphi][3])
            Int_FG0.append(output_array[ifile][ilayer][iphi][4])
            Int_BG0.append(output_array[ifile][ilayer][iphi][5])
            pt_arr0.append(output_array[ifile][ilayer][iphi][6])
            pt_arr_err0.append(output_array[ifile][ilayer][iphi][7])
        Means1.append(Means0)
        Mean_errs1.append(Mean_errs0)
        Sigmas1.append(Sigmas0)
        Sigma_errs1.append(Sigma_errs0)
        Int_FG1.append(Int_FG0)
        Int_BG1.append(Int_BG0)
        pt_arr1.append(pt_arr0)
        pt_arr_err1.append(pt_arr_err0)
    Means.append(Means1)
    Mean_errs.append(Mean_errs1)
    Sigmas.append(Sigmas1)
    Sigma_errs.append(Sigma_errs1)
    Int_FG.append(Int_FG1)
    Int_BG.append(Int_BG1)
    pt_arr.append(pt_arr1)
    pt_arr_err.append(pt_arr_err1)

In [12]:
types = ["emc_r_dphi_","emc_min_r_dphi_","emc_cnt_dphi_","emc_r_dz_","emc_min_r_dz_","emc_cnt_dz_"]
fit_functions = []
for ifile in range(N_files):
    fit_functions0 = []
    for ilayer in range(N_layers):
        fit_functions1 = []
        for iphi in range(N_phi):
            fit_functions1.append(root.TF1(f"fit_functions_sigma_{ifile}_{iphi}_{ilayer}","[0]+[1]*exp([2]*x)",0.5,5.0))
            fit_functions1[-1].SetParameter(0,0.0015)
            fit_functions1[-1].SetParameter(1,0.02)
            fit_functions1[-1].SetParameter(2,-3.3)
            fit_functions1[-1].SetParLimits(2,-8.126,-0.023)
            fit_functions1[-1].SetParLimits(0,0.001,0.002)
            if ilayer<2 : fit_functions1[-1].SetParLimits(2,-4.5,-2.5)
            elif ilayer<4 : fit_functions1[-1].SetParLimits(2,-4.0,-2.5)
            else : 
                fit_functions1[-1].SetParLimits(2,-6.0,-2.5)
                fit_functions1[-1].SetParLimits(1,0.001,0.01)
            if iOption0%2==1: 
                fit_functions1[-1].SetParLimits(2,-5.126,-1.23)
                fit_functions1[-1].SetParameter(1,0.02)
                fit_functions1[-1].SetParLimits(1,0.001,0.6)
                fit_functions1[-1].SetParameter(0,0.005)
                fit_functions1[-1].SetParLimits(0,0.001,0.006)
            if is_sigmalised: 
                fit_functions1[-1].SetParLimits(0,0.,20)
                fit_functions1[-1].SetParLimits(1,0.,20)
                fit_functions1[-1].SetParLimits(2,-30,+10.23)
            fit_functions1[-1].SetLineColor(colors[iphi])
        fit_functions0.append(fit_functions1)
    fit_functions.append(fit_functions0)

In [13]:
g_sigma = []
for ifile in range(N_files):
    g_sigma_f = []
    h2=[]
    c3 = root.TCanvas(f"c3{ifile}",f"c3{ifile}",1400,1800)
    c3.Divide(2,4)
    min, max = 0., 0.01
    if is_sigmalised==1: max, min = 1.99, 0.01
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for ilayer in range(N_layers):
        g_sigma_layer = []
        c3.cd(ilayer+1)
        h2.append(Format_Pad_old( 0.05, 5, min, max, "p_{T} (GeV/#it{c})", "Sigmas", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h2[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9,layers[ilayer]))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for iphi in range(0,N_phi):
            N_pt=len(pt_arr[ifile][ilayer][iphi])
            if N_pt<1: continue
            g_sigma_layer.append(root.TGraphErrors(N_pt, pt_arr[ifile][ilayer][iphi], Sigmas[ifile][ilayer][iphi],pt_arr_err[ifile][ilayer][iphi],Sigma_errs[ifile][ilayer][iphi]))
            g_sigma_layer[-1].SetName(f"sigma_o{iOption0}_s{sim}_f{ifile}_c{ilayer}_l{iphi}")
            Format_Graph(g_sigma_layer[-1],20+int(iphi/4),1, colors[iphi], 3, colors[iphi],1,1)
            if iOption0 == 0 and ilayer%2==0 and (iphi == 0 or iphi == N_phi-1): 
                g_sigma_layer[-1].Fit(fit_functions[ifile][ilayer][iphi],"Q","",0.2,1.2)
            else: g_sigma_layer[-1].Fit(fit_functions[ifile][ilayer][iphi],"Q","")
            g_sigma_layer[-1].Draw('P')
            central_bean_prev = round(0.002*phi_bins[2*iphi]-0.001-0.1,3)
            legends[-1].AddEntry(g_sigma_layer[-1],f"phi_prev {central_bean_prev}","p")
        legends[-1].Draw()
        g_sigma_f.append(g_sigma_layer)
    g_sigma.append(g_sigma_f)
    c3.Draw()
    c3.SaveAs(f"output/Sigmas/sigma_{ifile}{iOption0}{is_sigmalised}.png")

In [14]:
print(f"sigma_pt_params[{N_files}][{N_layers}][{N_phi}][3]="+"{")
for fit_functions_ifile in fit_functions:
    print("{")
    for fit_functions_ilayer in fit_functions_ifile:
        print("{")
        for fits in fit_functions_ilayer:
            print("{"+f"{round(fits.GetParameter(0),6)}, {round(fits.GetParameter(1),6)}, {round(fits.GetParameter(2),6)}"+"},")
        print("},") 
    print("},")      
print("};")        

sigma_pt_params[1][8][6][3]={
{
{
{0.00146, 0.005294, -4.499992},
{0.001282, 0.003913, -3.191829},
{0.001252, 0.003894, -3.111299},
{0.001264, 0.003865, -3.105044},
{0.00129, 0.003714, -3.084602},
{0.001468, 0.004956, -4.319149},
},
{
{0.00128, 0.003991, -3.252773},
{0.001258, 0.003926, -3.169525},
{0.001255, 0.003859, -3.135487},
{0.001263, 0.00392, -3.190264},
{0.001293, 0.00398, -3.310773},
{0.001449, 0.005262, -4.499973},
},
{
{0.001428, 0.007692, -3.889822},
{0.001335, 0.007611, -3.774011},
{0.001213, 0.006735, -3.300416},
{0.0012, 0.006684, -3.251662},
{0.00121, 0.006564, -3.219613},
{0.001375, 0.006991, -3.644637},
},
{
{0.001263, 0.006012, -3.186224},
{0.001205, 0.00626, -3.22299},
{0.001241, 0.006662, -3.454669},
{0.001257, 0.006854, -3.55027},
{0.001239, 0.006705, -3.466805},
{0.001348, 0.007434, -3.895833},
},
{
{0.001801, 0.01, -3.128795},
{0.001363, 0.009389, -2.734366},
{0.001296, 0.009605, -2.781013},
{0.001282, 0.009428, -2.744337},
{0.001255, 0.008912, -2.603634},
{0.0

In [15]:
fit_mean_functions = []
for ifile in range(N_files):
    fit_functions0 = []
    for ilayer in range(N_layers):
        fit_functions1 = []
        for iphi in range(N_phi):
            if not is_sigmalised: 
                fit_functions1.append(root.TF1(f"fit_functions_mean_{ifile}_{iphi}_"+layers[ilayer],"[0]",0.5,5.0))
                fit_functions1[-1].SetParameter(0,0.029)
            else: 
                fit_functions1.append(root.TF1(f"fit_functions_mean_{ifile}_{iphi}_"+layers[ilayer],"[0]+[1]*exp([2]*x)",0.0,5.0))
                if iOption0==0:
                    fit_functions1[-1].SetParLimits(2,-5.126,-0.23)
                    if ilayer%2==0: 
                        fit_functions1[-1].SetParameter(1,-1.05)
                        fit_functions1[-1].SetParLimits(1,-2.0,-1)
                        fit_functions1[-1].SetParameter(0,0.8)
                        fit_functions1[-1].SetParLimits(0,0.1,1.3)
                    else: 
                        fit_functions1[-1].SetParameter(1,1.05)
                        fit_functions1[-1].SetParLimits(1,1,2.0)
                        fit_functions1[-1].SetParameter(0,-0.8)
                        fit_functions1[-1].SetParLimits(0,-1.3,-0.1)
                    if iphi==3 and ilayer ==7:  
                        fit_functions1[-1].SetParameter(0,-0.9)
                        fit_functions1[-1].SetParameter(1,1.6)
                        fit_functions1[-1].SetParameter(2,-0.9)
            fit_functions1[-1].SetLineColor(colors[iphi])
        fit_functions0.append(fit_functions1)
    fit_mean_functions.append(fit_functions0)

In [16]:
for ifile in range(N_files):
    h2=[]
    c3 = root.TCanvas(f"c3{ifile}",f"c3{ifile}",1400,1800)
    c3.Divide(2,4)
    g_mean = [[[[]]*N_phi]*N_layers]*N_files
    min, max = -0.1, 0.1
    if is_sigmalised==1: max, min = 1.5, -1.5
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for ilayer in range(N_layers):
        c3.cd(ilayer+1)
        h2.append(Format_Pad_old( 0.05, 5, min, max, "p_{T} (GeV/#it{c})", "Means", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h2[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9,layers[ilayer]))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for iphi in range(0,N_phi):
            
            N_pt=len(pt_arr[ifile][ilayer][iphi])
            if N_pt<1: continue
            g_mean[ifile][ilayer][iphi].append(root.TGraphErrors(N_pt, pt_arr[ifile][ilayer][iphi], Means[ifile][ilayer][iphi],pt_arr_err[ifile][ilayer][iphi],Mean_errs[ifile][ilayer][iphi]))
            
            Format_Graph(g_mean[ifile][ilayer][iphi][-1],20+int(iphi/4),1, colors[iphi], 3, colors[iphi],1,1)
            g_mean[ifile][ilayer][iphi][-1].Draw('P')
            if not is_sigmalised: g_mean[ifile][ilayer][iphi][-1].Fit(fit_mean_functions[ifile][ilayer][iphi],"Q","",0.7,1)
            else: g_mean[ifile][ilayer][iphi][-1].Fit(fit_mean_functions[ifile][ilayer][iphi],"Q","",0.3,5)
            central_bean_prev = round(0.002*phi_bins[2*iphi]-0.001-0.1,3)
            legends[-1].AddEntry(g_mean[ifile][ilayer][iphi][-1],f"phi_prev {central_bean_prev}","p")

        legends[-1].Draw()
    c3.Draw()
    c3.SaveAs(f"output/Sigmas/mean_{ifile}{iOption0}{is_sigmalised}.png")

In [17]:
if not is_sigmalised:
    print(f"mean_pt_params[{N_files}][{N_layers}][{N_phi}]="+"{")
    for fit_functions_file in fit_mean_functions:
        print("{")
        for fit_functions_syst in fit_functions_file:
            print("{"+f"{round(fit_functions_syst[0].GetParameter(0),6)}, {round(fit_functions_syst[1].GetParameter(0),6)},{round(fit_functions_syst[2].GetParameter(0),6)}, {round(fit_functions_syst[3].GetParameter(0),6)}, {round(fit_functions_syst[4].GetParameter(0),6)}, {round(fit_functions_syst[5].GetParameter(0),6)}"+"},")
        print("},")
    print("};")
else:
    print("const float "+iOptames[iOption0]+f"_sMean_pt_params[{N_files}][{N_layers}][3]="+"{")
    for fit_functions_ifile in fit_mean_functions:
        print("{")
        iilayer = 0
        for fits in fit_functions_ifile:
            print("{"+f"{round(fits[3-int(iilayer%2)].GetParameter(0),6)}, {round(fits[3-int(iilayer%2)].GetParameter(1),6)}, {round(fits[3-int(iilayer%2)].GetParameter(2),6)}"+"},")
            iilayer+=1
        print("},")      
    print("};")        
    print(f"sigma_pt_params[{N_files}][{N_layers}][{N_phi}][3]="+"{")
    for fit_functions_ifile in fit_mean_functions:
        print("{")
        for fit_functions_ilayer in fit_functions_ifile:
            print("{")
            for fits in fit_functions_ilayer:
                print("{"+f"{round(fits.GetParameter(0),6)}, {round(fits.GetParameter(1),6)}, {round(fits.GetParameter(2),6)}"+"},")
            print("},") 
        print("},")      
    print("};")        

mean_pt_params[1][8][6]={
{
{-0.009817, -0.004736,-0.000661, 0.000351, 0.004422, 0.009508},
{-0.00945, -0.00438,-0.000327, 0.000683, 0.004735, 0.009798},
{-0.010466, -0.005309,-0.001183, -0.000148, 0.003982, 0.009129},
{-0.00948, -0.004302,-0.000154, 0.000882, 0.005026, 0.010194},
{-0.011723, -0.00624,-0.001712, -0.000575, 0.00395, 0.009544},
{-0.010165, -0.004492,1.7e-05, 0.00114, 0.005579, 0.01101},
{-0.01087, -0.00547,-0.001053, 5.6e-05, 0.004469, 0.009942},
{-0.010259, -0.004721,-0.000326, 0.000767, 0.005101, 0.010445},
},
};


In [18]:
dphi = array( 'd' )
dphi_err = array( 'd' )
dynamic_dphi = []
dynamic_dphi_err = []
for iphi in range(N_phi):
    if not is_sigmalised: dphi.append( round(0.001*phi_bins[2*iphi]-0.0005-0.1,5))
    else: dphi.append(round(0.2*phi_bins[2*iphi]-0.1-10,5))
    dphi_err.append(0.001/4)

for ifile in range(N_files):
    dynamic_dphi_ifile = []
    dynamic_dphi_err_ifile = []
    for ilayer in range(N_layers):
        dynamic_dphi_ilayer = array( 'd' )
        dynamic_dphi_err_ilayer = array( 'd' )
        for iphi in range(N_phi):
            dynamic_dphi_ilayer.append(fit_mean_functions[ifile][ilayer][iphi].GetParameter(0))
            dynamic_dphi_err_ilayer.append(fit_mean_functions[ifile][ilayer][iphi].GetParError(0))
        dynamic_dphi_ifile.append(dynamic_dphi_ilayer)
        dynamic_dphi_err_ifile.append(dynamic_dphi_err_ilayer)
    dynamic_dphi.append(dynamic_dphi_ifile)
    dynamic_dphi_err.append(dynamic_dphi_err_ifile)

print("dphi = ", dphi)
print(dynamic_dphi)
if iOption0 != 0:
    dynamic_dphi[0][5][2]/=100
    dynamic_dphi[0][5][3]=0.002
    dynamic_dphi[0][4][3]=-0.0005
    dynamic_dphi[0][4][4]/=5

dphi =  array('d', [-0.0095, -0.0045, -0.0005, 0.0005, 0.0045, 0.0095])
[[array('d', [-0.009817002635533052, -0.004735816054717217, -0.0006614349108440241, 0.00035067163008532104, 0.0044217728612059234, 0.009507569035673673]), array('d', [-0.009449918784498013, -0.004379698880000112, -0.0003269113748607354, 0.0006832645778392724, 0.004734778644996298, 0.009797765600934497]), array('d', [-0.01046644580013931, -0.005309113187782943, -0.0011825121283789881, -0.00014832581006481507, 0.003982260324892892, 0.009129476317418419]), array('d', [-0.00948009621249141, -0.004302372719345044, -0.000154106626031731, 0.0008821618772839358, 0.005025551729518227, 0.010194164351963936]), array('d', [-0.011723004125358, -0.006240268313746061, -0.0017115484019586714, -0.0005746996360558206, 0.003949943171763788, 0.00954352108747675]), array('d', [-0.010164906592328018, -0.004492280314605423, 1.719061155490767e-05, 0.0011399269891235742, 0.005579465414470377, 0.01101018531065473]), array('d', [-0.010869609

In [19]:
for ifile in range(N_files):
    h3=[]
    c4 = root.TCanvas(f"c4",f"c4",600,450)
    g_mean_phi = [[[[]]*N_phi]*N_layers]*N_files
    min, max, x0, x1 = -0.05, 0.05, -0.02, 0.05
    if is_sigmalised==1: max, min, x0, x1 = 2, -2, -2, 5
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for iphi in range(1):
        h3.append(Format_Pad_old( x0, x1, min, max, "phi_prev [rad]", "mean [rad]", 1.1, 1.1, 0.06, 0.05, "",0.2,0.2,0.,0.1))
        h3[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for ilayer in range(0,N_layers):

            g_mean_phi[ifile][ilayer][iphi].append(root.TGraphErrors(N_phi, dphi, dynamic_dphi[ifile][ilayer],dphi_err,dynamic_dphi_err[ifile][ilayer]))
            
            Format_Graph(g_mean_phi[ifile][ilayer][iphi][-1],20+int(ilayer/4),1, colors[ilayer], 3, colors[ilayer],1,1)
            g_mean_phi[ifile][ilayer][iphi][-1].Fit("pol1","Q","")
            g_mean_phi[ifile][ilayer][iphi][-1].GetFunction("pol1").SetLineColor(colors[ilayer])
            g_mean_phi[ifile][ilayer][iphi][-1].Draw('sameP')
            legends[-1].AddEntry(g_mean_phi[ifile][ilayer][iphi][-1],layers[ilayer],"p")

        legends[-1].Draw()
    c4.Draw()
    c4.SaveAs(f"output/Sigmas/mean_{ifile}{iOption0}{is_sigmalised}.png")

In [20]:
sigma_dynamic_dphi = []
sigma_dynamic_dphi_err = []
pt_point = 0.5

for ifile in range(N_files):
    dynamic_dphi_ifile = []
    dynamic_dphi_err_ifile = []
    for ilayer in range(N_layers):
        dynamic_dphi_ilayer = array( 'd' )
        dynamic_dphi_err_ilayer = array( 'd' )
        for iphi in range(N_phi):
            dynamic_dphi_ilayer.append(fit_functions[ifile][ilayer][iphi].Eval(pt_point)/(fit_functions[ifile][ilayer][2].Eval(pt_point)+fit_functions[ifile][ilayer][3].Eval(pt_point))*2)
            dynamic_dphi_err_ilayer.append((fit_functions[ifile][ilayer][iphi].GetParError(0)*2)/(fit_functions[ifile][ilayer][2].Eval(pt_point)+fit_functions[ifile][ilayer][3].Eval(pt_point))*2)
        dynamic_dphi_ifile.append(dynamic_dphi_ilayer)
        dynamic_dphi_err_ifile.append(dynamic_dphi_err_ilayer)
    sigma_dynamic_dphi.append(dynamic_dphi_ifile)
    sigma_dynamic_dphi_err.append(dynamic_dphi_err_ifile)

print(sigma_dynamic_dphi)
sigma_dynamic_dphi[0][5][0]=0.88

[[array('d', [0.9707172399458767, 0.9985915804618276, 0.9979588922113212, 1.002041107788679, 1.0030046987759278, 0.981489599773877]), array('d', [1.0029461907956865, 1.0017074386147702, 1.0003078006930508, 0.9996921993069492, 0.9971126077953827, 0.9732387714641364]), array('d', [1.0069295697983867, 0.9910919720460238, 0.9982889933777607, 1.0017110066222394, 1.004388889623312, 0.9976131780569221]), array('d', [1.0260056533582282, 1.0134790414490742, 1.0013770544444387, 0.9986229455555611, 1.000693231826916, 0.9940971991150968]), array('d', [1.0579036949058087, 1.0203539528481627, 1.002009471532962, 0.997990528467038, 0.9996789784030937, 0.9874418134550191]), array('d', [1.0063245828301048, 0.9999390706266966, 0.9986473849566964, 1.0013526150433036, 1.0104736182521532, 1.022033305891115]), array('d', [1.16643408624096, 1.0459832550036956, 1.002955934914763, 0.9970440650852371, 1.0001812716575627, 1.0288272464924701]), array('d', [1.0848378658714177, 1.0167397383056933, 0.992642974306375,

In [21]:
for ifile in range(N_files):
    h3=[]
    c4 = root.TCanvas(f"c4",f"c4",600,450)
    g_sigma_phi = [[[[]]*N_phi]*N_layers]*N_files
    min, max, x0, x1 = 0, 2, -0.02, 0.05
    if is_sigmalised==1: max, min, x0, x1 = 4, -2, -2, 5
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for iphi in range(1):
        h3.append(Format_Pad_old( x0, x1, min, max, "phi_prev [rad]", "sigma [rad]", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h3[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for ilayer in range(0,N_layers):

            g_sigma_phi[ifile][ilayer][iphi].append(root.TGraphErrors(N_phi, dphi, sigma_dynamic_dphi[ifile][ilayer],dphi_err,sigma_dynamic_dphi_err[ifile][ilayer]))
            
            Format_Graph(g_sigma_phi[ifile][ilayer][iphi][-1],20+int(ilayer/4),1, colors[ilayer], 3, colors[ilayer],1,1)
            g_sigma_phi[ifile][ilayer][iphi][-1].Fit("pol2","Q","",)
            g_sigma_phi[ifile][ilayer][iphi][-1].GetFunction("pol2").SetLineColor(colors[ilayer])
            g_sigma_phi[ifile][ilayer][iphi][-1].Draw('sameP')
            legends[-1].AddEntry(g_sigma_phi[ifile][ilayer][iphi][-1],layers[ilayer],"p")

        legends[-1].Draw()
    c4.Draw()
    c4.SaveAs(f"output/Sigmas/mean_{ifile}{iOption0}{is_sigmalised}.png")

In [22]:
print("const float "+iOptames[iOption0]+f"_sigma_pt_params[{N_files}][{N_layers}][3]="+"\n{")
for fit_functions_ifile in fit_functions:
    print("    {")
    for fits in fit_functions_ifile:
        print("        {"+f"{round((fits[2].GetParameter(0)+fits[3].GetParameter(0))*0.5,6)}, {round((fits[2].GetParameter(1)+fits[3].GetParameter(1))*0.5,6)}, {round((fits[2].GetParameter(2)+fits[3].GetParameter(2))*0.5,6)}"+"},")
    print("    },")      
print("};")
print("const float "+iOptames[iOption0]+f"_sigma_"+iOptames[iOption0]+f"_params[{N_files}][{N_layers}][3]="+"\n{")
for ifile in range(N_files):
    print("    {")
    for ilayer in range(N_layers):
        print("        {"+f'{round(g_sigma_phi[ifile][ilayer][0][ilayer].GetFunction("pol2").GetParameter(0),6)}, {round(g_sigma_phi[ifile][ilayer][0][ilayer].GetFunction("pol2").GetParameter(1),6)}, {round(g_sigma_phi[ifile][ilayer][0][ilayer].GetFunction("pol2").GetParameter(2),6)}'+"},")
    print("    },")
print("};")
print("const float "+iOptames[iOption0]+f"_mean_"+iOptames[iOption0]+f"_params[{N_files}][{N_layers}][2]="+"\n{")
for ifile in range(N_files):
    print("    {")
    for ilayer in range(N_layers):
        print("        {"+f'{round(g_mean_phi[ifile][ilayer][0][ilayer].GetFunction("pol1").GetParameter(0),6)}, {round(g_mean_phi[ifile][ilayer][0][ilayer].GetFunction("pol1").GetParameter(1),6)}'+"},")
    print("    },")
print("};")



const float phi_sigma_pt_params[1][8][3]=
{
    {
        {0.001258, 0.00388, -3.108172},
        {0.001259, 0.00389, -3.162876},
        {0.001207, 0.00671, -3.276039},
        {0.001249, 0.006758, -3.502469},
        {0.001289, 0.009517, -2.762675},
        {0.00123, 0.0088, -2.653196},
        {0.001183, 0.008294, -5.018492},
        {0.001212, 0.009256, -5.377755},
    },
};
const float phi_sigma_phi_params[1][8][3]=
{
    {
        {1.002601, 0.640418, -274.897374},
        {1.000945, -1.360583, -142.572484},
        {0.99929, 0.593643, 11.823185},
        {1.002593, -1.61017, 100.735007},
        {1.002872, -2.60676, 225.139321},
        {1.007411, 3.270938, -657.580765},
        {1.001559, -5.586294, 956.614554},
        {0.998656, 2.416963, 1263.49999},
    },
};
const float phi_mean_phi_params[1][8][2]=
{
    {
        {-0.000156, 1.017157},
        {0.000177, 1.012972},
        {-0.000666, 1.031567},
        {0.000361, 1.0357},
        {-0.001126, 1.121847},
        {0.000515

In [23]:
outrootfile = root.TFile(f"output/dynamic_delta_fits/dynamicdeltasigma{iOption0}{sim}.root","RECREATE")
for g_sigma_f in g_sigma:
    for g_sigma_layer in g_sigma_f:
        for g_sigma_phi in g_sigma_layer:
            g_sigma_phi.Write()